In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('DFT.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_MC_O1         68 non-null     float64
 4   Cate_MC_H1         68 non-null     float64
 5   Cate_MC_O2         68 non-null     float64
 6   Cate_MC_H2         68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  P_name             68 non-null     object 
 13  P_HOMO             68 non-null     float64
 14  P_LUMO             68 non-null     float64
 15  P_MC_P             68 non-null     float64
 16  P_MC_=O            68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_MC_O1  Cate_MC_H1  Cate_MC_O2  Cate_MC_H2  \
0  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
1  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
2  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
3  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
4  -5.807916  -0.157824   -0.376813    0.263887   -0.418652    0.276114   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  Cate_total_dipole  \
0       1.36318       0.96622       1.37543       0.96253             2.8219   
1       1.36318       0.96622       1.37543       0.96253             2.8219   
2       1.36318       0.96622       1.37543       0.96253             2.8219   
3       1.36318       0.96622       1.37543       0.96253             2.8219   
4       1.36386       0.96625       1.37631       0.96239             2.5961   

     P_HOMO    P_LUMO    P_MC_P   P_MC_=O    P_MC_

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []
loo_score = []

#knn
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    
    param_knn={'n_neighbors':[1,2,3,4,5,6,7,8,9,10], 'p':[1,2,3,4,5]}
    reg_knn = GridSearchCV(KNeighborsRegressor(), param_grid=param_knn, cv=5, n_jobs=16)
    reg_knn.fit(a_X_train,y_train['Yield'])
    reg_best = reg_knn.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)
    y_pred2 = reg_best.predict(a_X_test)

    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    parametors = reg_knn.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_knn.best_score_
    data_cv_score.append(cv_score)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_knn.best_params_)
    print("Best Model Score:",reg_knn.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'n_neighbors': 4, 'p': 1}
Best Model Score: 0.7078076873126546
R2_test: 0.639077631820751
----------------------
seed: 1
Best Model Parameter: {'n_neighbors': 1, 'p': 1}
Best Model Score: 0.6826568635836722
R2_test: 0.32384110920231635
----------------------
seed: 2
Best Model Parameter: {'n_neighbors': 4, 'p': 1}
Best Model Score: 0.680060801207395
R2_test: 0.538328831300384
----------------------
seed: 3
Best Model Parameter: {'n_neighbors': 3, 'p': 1}
Best Model Score: 0.5339347960003863
R2_test: 0.772614007232954
----------------------
seed: 4
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.6197252980919629
R2_test: 0.6767092706068153
----------------------
seed: 5
Best Model Parameter: {'n_neighbors': 5, 'p': 1}
Best Model Score: 0.6155252467746972
R2_test: 0.6361340627132487
----------------------
seed: 6
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.6144025469120589
R2_test: 0.67814

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/DFT/knn_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.803077   12.049267   7.547872  0.639078  21.727136  14.011905   
1     1.000000    0.000000   0.000000  0.323841  25.594828  18.142857   
2     0.833067   12.426968   7.420213  0.538329  21.129702  16.833333   
3     0.782514   14.242579   8.567376  0.772614  14.772705  10.857143   
4     0.717841   15.596422  10.908815  0.676709  18.540843  12.707483   
5     0.752817   14.147551   9.063830  0.636134  21.036522  15.285714   
6     0.772247   13.671531   9.379433  0.678144  19.680988  13.412698   
7     0.925637    8.433722   5.255319  0.316700  24.888083  17.738095   
8     0.765704   14.359925   9.712766  0.728915  16.367524  11.107143   
9     0.789277   14.613550  10.138298  0.742914  13.459485  10.083333   
mean  0.814218   11.954151   7.799392  0.605338  19.719782  14.017971   
std   0.077583    4.206611   2.876156  0.148233   3.634088   2.635705   

      cv_score  
0     0.707808  
1     0.682657  